In [1]:
import numpy as np
import os
import sys
sys.path.append("C://Users//User//PycharmProjects//Musgrave_scripts//BenRich//common_funcs")
import cohpfuncs as cfunc
import jdftxfuncs as jfunc
import poscarclass as Poscar

In [2]:
test = Poscar.Poscar("D://data//aziz//BEAST//MN4C//pyridinic//backup//adsorbed//Pt//H//-1.00V//01//CONTCAR")
test.dump_jdftx_ionpos()
test.dump_jdftx_lattice()

In [3]:
test = Poscar.Poscar("D://data//aziz//BEAST//MN4C//pyridinic//backup//adsorbed//Pt//H//0.00V//01//CONTCAR")
test.dump_jdftx_ionpos()
test.dump_jdftx_lattice()

In [4]:
test = Poscar.Poscar("D://data//aziz//BEAST//MN4C//pyridinic//backup//adsorbed//Pt//H//No_bias//01//CONTCAR")
test.dump_jdftx_ionpos()
test.dump_jdftx_lattice()

In [5]:
test = Poscar.Poscar("D://data//aziz//BEAST//MN4C//pyridinic//backup//adsorbed//Pt//H//-0.50V//01//CONTCAR")
test.dump_jdftx_ionpos()
test.dump_jdftx_lattice()

In [20]:
in_checks = {
    "elec-cutoff": [None, "elec-cutoff 20 100"],
    "elec-n-bands": [None, "elec-n-bands foo"],
    "kpoint-folding": [None, "kpoint-folding 3 3 1"],
    "target-mu": [None, "#target-mu foo"],
    "latt-move-scale": [None, "latt-move-scale 0 0 0"],
    "lattice-minimize": [None, "lattice-minimize nIterations 0"],
    "ionic-minimize": [None, "ionic-minimize nIterations 0"]
}

elec_checks = {
    "elec-ex-corr": [None, "elec-ex-corr gga"],
    "van-der-waals": [None, "van-der-waals D3"],
    "symmetries": [None, "symmetries none" ],
    "electronic-minimize": [None, "electronic-minimize nIterations 100 energyDiffThreshold 1e-07"],
    "elec-smearing": [None, "elec-smearing Fermi 0.001"],
    "spintype": [None, "spintype z-spin"],
    "core-overlap-check": [None, "core-overlap-check none"],
    "elec-initial-magnetization": [None, "elec-initial-magnetization 0 no"],
    "coulomb-interaction": [None, "coulomb-interaction slab 001"],
    "converge-empty-states": [None, "converge-empty-states yes"]
}

solv_checks = {
    "fluid": [None, "fluid LinearPCM"],
    "pcm-variant": [None, "pcm-variant CANDLE"],
    "fluid-solvent": [None, "fluid-solvent H2O"],
    "fluid-cation": [None, "fluid-cation Na+ 0.5" ],
    "fluid-anion": [None, "fluid-anion F- 0.5"]
}

dump_checks = {
    "dsfg": [None, "dump End State"],
    "dsf": [None, "dump End Ecomponents"],
    "hgg": [None, "dump End EigStats"],
    "ghtgh": [None, "dump End BandEigs"],
    "gtgghf": [None, "dump End BandProjections"],
    "gffgf": [None, "dump End Gvectors"]
}

line_and_checks = {
    "in": in_checks,
    "elec": elec_checks,
    "solv": solv_checks,
    "dump": dump_checks
}

def read_inputs(fpath, line_and_checks):
    check_keys = line_and_checks.keys()
    with open(fpath, 'r') as f:
        for line in f:
            for check in check_keys:
                if check in line:
                    line_and_checks[check][0] = line
    return line_and_checks

def _get_param_dump_str(line_and_checks, key):
    dump_str = ""
    for input in line_and_checks[key].keys():
        if line_and_checks[key][input][0] is None:
            dump_str += line_and_checks[key][input][1] + " \n"
        else:
            dump_str += line_and_checks[key][input][0] + " \n"
    return dump_str

def write_params(data_path, line_and_checks, key):
    dump_str = _get_param_dump_str(line_and_checks, key)
    with open(data_path + "params." + key, "w") as f:
        f.write(dump_str)

def write_all_params(data_path, line_and_checks):
    for key in line_and_checks.keys():
        if key != "in":
            write_params(data_path, line_and_checks, key)


def _get_in_dump_str(line_and_checks, poscar):
    dump_str = "dump-name output.$VAR \n \n "
    keys = line_and_checks["in"].keys()
    for key in keys:
        if line_and_checks["in"][key][0] is None:
            dump_str += line_and_checks["in"][key][1]
        else:
            dump_str += line_and_checks["in"][key][0]
    dump_str += "\n \n"
    for atom in poscar.atom_names:
        dump_str += "ion-species GBRV/" + atom.lower() + "_pbe_v1.uspp" + "\n"
    dump_str += "include params.lattice \n include params.ionpos \n \n"
    ins = line_and_checks.keys()
    for key in ins:
        if key != "in":
            dump_str += "include params." + key + " \n"
    return dump_str

################################################################

def gc_man_out_to_perl_in(data_path, line_and_checks, save_path = None):
    if save_path is None:
        save_path = data_path
    poscar = Poscar.Poscar(data_path + "CONTCAR")
    inpath = data_path + "inputs"
    for check in line_and_checks.keys():
        line_and_checks[check] = read_inputs(inpath, line_and_checks[check])
    with open(save_path + "in", 'w') as f:
        f.write(_get_in_dump_str(line_and_checks, poscar))
    write_all_params(save_path, line_and_checks)

def _get_submit_dump_str():
    dump_str = ""
    dump_str += "#!/bin/bash \n"
    dump_str += "#SBATCH -J jobnamevar \n"
    dump_str += "#SBATCH -A m4025_g \n"
    dump_str += "#SBATCH -C gpu \n"
    dump_str += "#SBATCH -q regular_ss11 \n"
    dump_str += "#SBATCH -t 15:00 \n"
    dump_str += "#SBATCH -N 1 \n#SBATCH -n 2 \n#SBATCH --ntasks-per-node=2 \n#SBATCH -c 32 \n#SBATCH --gpus-per-task=1 \n#SBATCH --gpu-bind=none \n#SBATCH -o %j.out \n#SBATCH -e %j.err \n \n"
    dump_str += "module use /global/homes/b/beri9208/modules \nmodule load jdftx/gpu \n \n"
    dump_str += "export SLURM_CPU_BIND=\"cores\" \nexport JDFTX_MEMPOOL_SIZE=8192 \n \nsrun jdftx_gpu -i in -o out \n"
    return dump_str

def write_submit(data_path):
    dump_str = _get_submit_dump_str()
    with open(data_path + "submit.sh", "w", newline='\n') as f:
        f.write(dump_str)

In [22]:
biases = ["No_bias", "0.00V", "-0.50V", "-1.00V"]
elmts = ["Fe", "Ni", "Y", "Zn", "Pt"]
for e in elmts:
    for b in biases:
        data_path = "D://data//aziz//BEAST//MN4C//pyridinic//backup//adsorbed//" + e + "//H//" + b + "//01//"
        save_path = "E://perl//pscratch//beri9208//HydrideTest//AzizStruc//No_opt_psuedo_v1//" + e + "//" + b + "//"
        poscar = Poscar.Poscar(data_path + "CONTCAR")
        poscar.dump_jdftx_ionpos()
        poscar.dump_jdftx_lattice()
        gc_man_out_to_perl_in(data_path, line_and_checks, save_path=save_path)
        write_submit(save_path)

In [ ]:
elmts = [["H", "He"],
         ["Li", "Be", "B", "C", "N", "O", "F", "Ne"],
         ["Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar"],
         ["K", "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr"],
         ["Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe"],
         ]






In [7]:
test.atom_names[0].lower()

'c'